## Installation and Setup

In [1]:
from pathlib import Path
import os, sys

# Make this path be the project's "base" directory, so we can include modules
notebook_directory_ancestor = Path.cwd().resolve().parent.parent.parent
print(notebook_directory_ancestor)
core_code_directory = notebook_directory_ancestor / "PythonLibraries" / "CoreCode"

is_core_code_directory_in_sys_path = str(core_code_directory) in sys.path
is_notebook_directory_ancestor_in_sys_path = str(notebook_directory_ancestor) in sys.path
print("Is CoreCode directory in sys.path?", is_core_code_directory_in_sys_path)
print("Is notebook directory's ancestor in sys.path?", is_notebook_directory_ancestor_in_sys_path)

if not is_core_code_directory_in_sys_path:
    sys.path.append(str(core_code_directory))

/InServiceOfX
Is CoreCode directory in sys.path? False
Is notebook directory's ancestor in sys.path? False


In [2]:
from corecode.FileIO.get_filepaths import get_filepaths
from corecode.Utilities import load_environment_file
load_environment_file()
from corecode.Utilities import (
    DataSubdirectories,)

In [3]:
import nest_asyncio

nest_asyncio.apply()

### Setup LLM using Groq 

In [4]:
from llama_index.llms.groq import Groq

In [10]:
llm = Groq(
    api_key=os.environ["GROQ_CLOUD_API_KEY"],
    model="llama3-8b-8192")
llm_70b = Groq(
    api_key=os.environ["GROQ_CLOUD_API_KEY"],
    model="llama3-70b-8192")

### Setup Embedding Model

In [6]:
from llama_index.embeddings.huggingface import HuggingFaceEmbedding

embed_model = HuggingFaceEmbedding(model_name="BAAI/bge-small-en-v1.5")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


In [7]:
print(type(embed_model))
print(type(llm))
print(type(llm_70b))

# I wanted to try loading another model I can find on HuggingFace
embed_model_allmini = HuggingFaceEmbedding(model_name="sentence-transformers/all-MiniLM-L6-v2")

<class 'llama_index.embeddings.huggingface.base.HuggingFaceEmbedding'>
<class 'llama_index.llms.groq.base.Groq'>
<class 'llama_index.llms.groq.base.Groq'>


### Define Global Settings Configuration

In LlamaIndex, you can define global settings so you don't have to pass the LLM / embedding model objects everywhere.

In [11]:
from llama_index.core import Settings

Settings.llm = llm
Settings.embed_model = embed_model
print(type(Settings))
print(type(Settings.llm))
print(type(Settings.embed_model))

<class 'llama_index.core.settings._Settings'>
<class 'llama_index.llms.groq.base.Groq'>
<class 'llama_index.embeddings.huggingface.base.HuggingFaceEmbedding'>


### Load Data

We load data using LlamaParse by default, but you can also choose to opt for our free pypdf reader (in SimpleDirectoryReader by default) if you don't have an account.

In [9]:
from llama_parse import LlamaParse

data_sub_dirs = DataSubdirectories()

pdf_path_1 = data_sub_dirs.PublicFinances / "Taxes" / "2023" / "i1040gi.pdf"
pdf_path_2 = data_sub_dirs.PublicFinances / "Taxes" / "2023" / "i1040sc.pdf"
pdf_path_3 = data_sub_dirs.PublicFinances / "Taxes" / "2023" / "i1040tt.pdf"
print(pdf_path_1.exists())
print(pdf_path_2.exists())
print(pdf_path_3.exists())
doc_1040_general_instructions = LlamaParse(
    api_key=os.environ["LLAMA_CLOUD_API_KEY"],
    result_type="text").load_data(pdf_path_1)
doc_1040_schedule_c = LlamaParse(
    api_key=os.environ["LLAMA_CLOUD_API_KEY"],
    result_type="text").load_data(pdf_path_2)
doc_1040_tax_tables = LlamaParse(
    api_key=os.environ["LLAMA_CLOUD_API_KEY"],
    result_type="text").load_data(pdf_path_3)

True
True
True
Started parsing the file under job_id 19b800c5-13bb-4ee0-9a7a-d0ccff010403
Started parsing the file under job_id 540f148a-26f5-4a9d-acc6-faf9d8d8f4fb
Started parsing the file under job_id 8f99c524-b0c7-45c1-bf23-c49b532b1934


## 1. Basic Completion and Chat 

### Call complete with a prompt

In [13]:
response = llm.complete("Give me a summary of how to fill out a Schedule C and what things I can expense as business and count as an expense")
print(type(response))

<class 'llama_index.core.base.llms.types.CompletionResponse'>


In [14]:
print(response)

The Schedule C (Form 1040) is a tax form used to report the income and expenses of a sole proprietorship or single-member limited liability company (LLC). Here's a summary of how to fill it out and what expenses you can deduct as a business expense:

**Step 1: Gather necessary documents**

* Business income statements (e.g., invoices, receipts, bank statements)
* Business expense records (e.g., receipts, invoices, cancelled checks)
* Business-related documents (e.g., contracts, agreements, licenses)

**Step 2: Complete the header information**

* Enter your name, address, and Social Security number or Employer Identification Number (EIN)
* Check the box indicating you're a sole proprietor or single-member LLC

**Step 3: Report business income**

* List all business income on Line 1, including:
	+ Sales revenue
	+ Services rendered
	+ Interest income
	+ Dividend income
	+ Other income (e.g., rent, royalties)

**Step 4: Report business expenses**

* List all business expenses on Schedule

In [16]:
stream_response = llm.stream_complete(
    """ You're originally a hardware level, GPU software engineer, who now is trying to start a business that 
    would help fund his personal research and development and lifestyle. What can he expense in his business?
    Can he expense GPU depreciation? """
)

print(type(stream_response))
for t in stream_response:
    print(t.delta, end="")
print(type(t))

<class 'generator'>
As a hardware-level GPU software engineer turned entrepreneur, it's exciting to explore the possibilities of expensing business expenses to fund your personal research and development, as well as your lifestyle. Here's a breakdown of what you can and can't expense:

**Expensible items:**

1. **Office expenses:** Rent, utilities, internet, and other miscellaneous expenses related to your home office or a dedicated workspace.
2. **Software and tools:** Licenses, subscriptions, and tools necessary for your research and development, such as:
	* Development environments (e.g., IDEs, compilers)
	* Version control systems (e.g., Git)
	* Debugging and profiling tools
	* Research papers and academic journals
3. **Travel and education:** Expenses related to attending conferences, workshops, and online courses to stay updated on the latest developments in your field.
4. **Equipment and hardware:** Expenses related to purchasing or upgrading your personal computer, peripherals,

### Call chat with a list of messages

In [ ]:
from llama_index.core.llms import ChatMessage

messages = [
    ChatMessage(
        role="system", content),
    
]